# Edmonton Open Data - Property Assessment Map Analysis

This tool would best serve the Edmonton residential property owner population. Providing insight into the range and tends of property assessments. The final cell of this Notebook provides an interactive solution for exploring the dataset. The tooltip can be optimized with various pieces of relevant statistics or potentially visualizations.

In [1]:
# import the necessary packages
import pandas as pd
import folium

In [2]:
# read in the csv to a new DataFrame: property_assess
property_assess = pd.read_csv('assessment_per_capita.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# define the mean latitude and longitude for the dataset: yeg_lat & yeg_long
yeg_lat = property_assess['lat'].mean()
yeg_long = property_assess['long'].mean()

# print the mean values
print(yeg_lat, yeg_long)

53.52064058590727 -113.50898651489592


In [4]:
# generate a new folium map centered on the datasets mean lat,long values
# the starting zoom level is selected through trial and error
m = folium.Map(location=[yeg_lat, yeg_long], zoom_start=10)
m

In [5]:
# define a function that will map the results for a particular neighbourhood
def nb_map(neighbourhood):
    ''' This function takes a neighbourhood as input and returns a folium map
    with a colour map associated with the property assessment value and a
    on-hover tooltip with assessment details.
    '''
    
    my_nb = property_assess[property_assess['nb'] == str(neighbourhood).upper()]

    m2 = folium.Map(location=[my_nb['lat'].mean(), my_nb['long'].mean()], zoom_start=15)
    
    def add_marker(row):
        if row['value'] < 100000:
            color = '#bfd7ff'
        elif row['value'] < 200000:
            color = '#a4c4fc'
        elif row['value'] < 300000:
            color = '#89b4ff'
        elif row['value'] < 400000:
            color = '#669dff'
        elif row['value'] < 500000:
            color = '#3f84ff'
        elif row['value'] < 600000:
            color = '#1e6fff'
        elif row['value'] < 700000:
            color = '#005bff'
        elif row['value'] < 800000:
            color = '#0045c6'
        elif row['value'] < 900000:
            color = '#012f87'
        elif row['value'] < 1000000:
            color = '#002263'
        elif row['value'] < 1100000:
            color = '#001338'
        elif row['value'] < 120000:
            color = '#820000'
        elif row['value'] < 1300000:
            color = '#b70000'
        elif row['value'] < 1400000:
            color = '#db0000'
        else:
            color = '#ff0000'

        row_cur = '${:0,.2f}'.format(row['value'])        
        tooltip = '<p><b>Address:</b></p><p>' + str(row['number']) + " " + row['street'] \
                  + '</p>' \
                  + '<p><b>Assessed Value:</b></p><p>' + row_cur + '</p>'
        marker = folium.CircleMarker([row['lat'], row['long']], radius=2, 
                                         color=color, tooltip=tooltip)
        marker.add_to(m2)

    my_nb.apply(add_marker, axis=1)

    return m2

In [6]:
nb_map('starling')

In [7]:
# generate a list of unique neighbourhoods to use in
# a dropdown Jupyter widget
nb_list = list(property_assess['nb'].unique())
nb_list = sorted(nb_list)

In [8]:
# import the necessary packages
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [9]:
_ = interact(nb_map, neighbourhood=nb_list)

interactive(children=(Dropdown(description='neighbourhood', options=('ABBOTTSFIELD', 'ALBANY', 'ALBERTA AVENUE…